In [1]:
import pandas as pd
import numpy as np
import re
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import style
style.use('ggplot')

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

In [2]:
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
from wordcloud import WordCloud

In [5]:
data = pd.read_csv("D://Req_Model_Deploy/Data/Labelled_Data.csv",encoding= "latin1")

# data = pd.read_csv('D:/RequiremetsMgmnt/Status_model/Labelled_Data.csv')

FileNotFoundError: [Errno 2] No such file or directory: '/home/dheerus/Desktop/Flask_ML_Deploy/D://Req_Model_Deploy/Data/Labelled_Data.csv'

In [ ]:
data['Requirements'].isna().sum()

12

In [ ]:
import re
from nltk.tokenize import word_tokenize

def data_processing(Requirements):
    if isinstance(Requirements, str):
        Requirements = Requirements.lower()
        Requirements = re.sub(r"https\S+|www\S+http\S+", '', Requirements, flags=re.MULTILINE)
        Requirements = re.sub(r'[^\w\s]', '', Requirements)  # Remove all non-word characters
        Requirements_tokens = word_tokenize(Requirements)
        # print(Requirements_tokens)
        return " ".join(Requirements_tokens)
    else:
        return str(Requirements)


# def preprocess_text(text):
#     if isinstance(text, str):
#         return text
#     elif pd.isna(text):
#         return ""
#     else:
#         return str(text)

# # Apply preprocessing to 'requirements' column
# data['Requirements'] = data['Requirements'].apply(preprocess_text)

In [ ]:
data["processed_requirements"] = data["Requirements"].apply(data_processing)

In [ ]:
data["processed_requirements"].isna().sum()

0

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MaxAbsScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data['processed_requirements'], data['Labels'], test_size=0.2, random_state=42)

# Vectorize the text data
vectorizer = TfidfVectorizer()
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

# Apply MaxAbsScaler to TF-IDF vectors
scaler = MaxAbsScaler()
X_train_scl = scaler.fit_transform(X_train_vec)
X_test_scl = scaler.transform(X_test_vec)

# Create and train the logistic regression model
log_reg = LogisticRegression()
log_reg.fit(X_train_scl, y_train)

# Calculate training accuracy
log_train_accuracy = log_reg.score(X_train_scl, y_train)
print("Training accuracy:", log_train_accuracy*100)

# Make predictions on the test set
log_pred = log_reg.predict(X_test_scl)

# Calculate accuracy
log_accuracy = accuracy_score(y_test, log_pred)
print("Accuracy for Logistic Regression Model is:", log_accuracy*100)


########################################################
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
param_grid = {'C': [100, 10, 1.0, 0.1, 0.01], 'solver': ['newton-cg', 'lbfgs', 'liblinear']}

# Create the GridSearchCV object
grid = GridSearchCV(LogisticRegression(), param_grid, cv=5)
grid.fit(X_train_scl, y_train)
grid_train_accuracy = grid.score(X_train_scl, y_train)
print("Grid Training accuracy:", log_train_accuracy*100)
y_pred_grid = grid.predict(X_test_scl)
grid_accuracy = accuracy_score(y_test, y_pred_grid)
print("Accuracy for Grid CV Regression Model is:", grid_accuracy*100)


#########################################################

from sklearn.ensemble import RandomForestClassifier

model_rf = RandomForestClassifier()
model_rf.fit(X_train_scl, y_train)

y_pred_random = model_rf.predict(X_test_scl)

random_Accuracy = accuracy_score(y_pred_random, y_test)

print("Random Training Accuracy :", model_rf.score(X_train_scl, y_train)*100)
print("Random Testing Accuracy :{:.2f}%".format(random_Accuracy*100))



##########################################

# from sklearn.tree import DecisionTreeClassifier

# model_dt = DecisionTreeClassifier()
# model_dt.fit(X_train_scl, y_train)

# y_preds_dt = model_dt.predict(X_test_scl)

# dt_Accuracy = accuracy_score(y_preds_dt, y_test)

# print("DT Training Accuracy :", model_dt.score(x_train, y_train)*100)
# print("DT Testing Accuracy :{:.2f}%".format(dt_Accuracy*100))

Training accuracy: 97.12203557312253
Accuracy for Logistic Regression Model is: 90.9090909090909
Grid Training accuracy: 97.12203557312253
Accuracy for Grid CV Regression Model is: 91.40316205533597
Random Training Accuracy : 98.40662055335969
Random Testing Accuracy :90.07%


In [ ]:
import pickle
# pickle.dump(vectorizer, open('D:/Req_Model_Deploy/Models/countVectorizer.pkl', 'wb'))
# pickle.dump(scaler, open('D:/Req_Model_Deploy/Models/scaler.pkl', 'wb'))
# pickle.dump(grid, open('D:/Req_Model_Deploy/Models/grid.pkl', 'wb'))

# pickle.dump(grid, open('D:/Req_Model_Deploy/Models/grid.pkl', 'wb'))



In [ ]:
/home/dheerus/Desktop/Flask_ML_Deploy/Models/Type_model_grid.pkl

In [3]:
import pickle

vectorizer = pickle.load(open('Models/countVectorizer.pkl', 'rb'))
scaler = pickle.load(open('Models/scaler.pkl', 'rb'))
grid = pickle.load(open('Models/Type_model_grid.pkl', 'rb'))

In [ ]:

import pandas as pd
vect = TfidfVectorizer()
# Try reading the CSV file with different encodings
# encodings = ['utf-8', 'latin1', 'iso-8859-1', 'cp1252']
# for encoding in encodings:
#     try:
#         data = pd.read_csv('Data/Labelled_Data.csv', encoding=encoding)
#         print("CSV file read successfully with encoding:", encoding)
#         break  # Stop trying other encodings if successful
#     except UnicodeDecodeError:
#         print("Failed to read with encoding:", encoding)

# Read the file containing requirements
file_path = "D:/RequiremetsMgmnt/Data/Labelled_Data.xlsx"  # Change this to the path of your file
data = pd.read_excel(file_path)

# Assuming your file has a column named 'Requirements' containing the text of requirements
# If your file has a different column name, replace 'Requirements' with your column name
requirements = data['Requirements']
requirements.apply(data_processing)

X = vectorizer.fit_transform(requirements)

# Create a list to store predictions
predictions = []

# Iterate through each requirement
for req_transformed in X:
    # Transform the requirement using your vectorizer
    # req_transformed = vect.transform([req]).toarray()
    
    # Predict the output
    output = grid.predict(req_transformed)
    
    # Append the prediction to the predictions list
    predictions.append(output[0])  # Assuming grid.predict returns an array, so taking the first element

# Add predictions as a new column to the DataFrame
data['Model_Predictions'] = predictions

# # Print the DataFrame with predictions
# print(data)
import os 

user_home = os.path.expanduser("~")

output_folder = os.path.join(user_home, "RequiremetsMgmnt", "predictions")
os.makedirs(output_folder, exist_ok=True)
output_file_path = os.path.join(output_folder, "Test_flask_deploy.csv")
data.to_csv(output_file_path, index=False)
# # Save the modified DataFrame to a new CSV file
# output_file_path = "D:/RequiremetsMgmnt/predictions"  # Change this to the path where you want to save the file
# data.to_csv(output_file_path, index=False)

# Provide a link for downloading the file
print("File has been saved. Click the link below to download:")
print(output_file_path)

In [4]:
def preprocess_input(input_text):
    # Vectorize input text
    vectorized_text = vectorizer.transform([input_text])
    # Scale vectorized text
    scaled_text = scaler.transform(vectorized_text)
    return scaled_text

def predict(input_text):
    # Preprocess input
    preprocessed_input = preprocess_input(input_text)
    # Make predictions
    predictions = grid.predict(preprocessed_input)
    return predictions


In [7]:
input_text = "The number of DUT to be tested are 3"
predictions = predict(input_text)
print(predictions)


['Technical - Testing']


In [1]:
import flask

flask.__version__

/tmp/ipykernel_14321/2164524030.py:3: DeprecationWarning: The '__version__' attribute is deprecated and will be removed in Flask 3.1. Use feature detection or 'importlib.metadata.version("flask")' instead.
  flask.__version__


'3.0.3'

In [2]:
import werkzeug

werkzeug.__version__




/tmp/ipykernel_14321/3409301833.py:3: DeprecationWarning: The '__version__' attribute is deprecated and will be removed in Werkzeug 3.1. Use feature detection or 'importlib.metadata.version("werkzeug")' instead.
  werkzeug.__version__


'3.0.3'